If running on google colab, uncomment and run the first cell 
Otherwise, build environment from Dockerfile with access to nvidia gpu 

In [ ]:
#@title Setup your linux environment on Google Colab

# !sudo apt install python3.8-full python3-pip
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
# !python --version

# !pip3 install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121
# !pip install -U openmim xtcocotools munkres ffmpeg-python ipykernel moviepy scikit-video gdown && \
# !mim install mmengine && \
# !pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html && \
# !mim install "mmdet>=3.1.0" 

In [ ]:
import os  

if not os.path.exists('mmpose'):
    !git clone https://github.com/open-mmlab/mmpose.git

%cd mmpose

!pip install -r requirements.txt
!pip install -v -e .

!mim install "mmpose>=1.1.0" --quiet
!mim install "mmpretrain>=1.0.0rc8" #needed for video inference with pretrained models

In [ ]:
#@title Check that your install is working

!mim download mmpose --config td-hm_hrnet-w48_8xb32-210e_coco-256x192  --dest .

!python demo/image_demo.py \
    tests/data/coco/000000000785.jpg \
    td-hm_hrnet-w48_8xb32-210e_coco-256x192.py \
    td-hm_hrnet-w48_8xb32-210e_coco-256x192-0e67c616_20220913.pth \
    --out-file vis_results.jpg \
    --draw-heatmap


# show image
from IPython.display import Image
Image("vis_results.jpg")

In [ ]:
#@title Now let's try inference on a video
from IPython.display import Video

video_file = "./demo/resources/demo.mp4"
pred_out_dir = "./demo/pred_out"
vis_out_dir = "./demo/vis_out"

os.makedirs(pred_out_dir, exist_ok=True)
os.makedirs(vis_out_dir, exist_ok=True)

!python ./demo/inferencer_demo.py {video_file} \
    --pose2d vitpose-h --pred-out-dir {pred_out_dir} \
    --vis-out-dir {vis_out_dir} --skeleton-style openpose --radius 5 --thickness 3 #--black-background= f""

# Path to your video file
Video(f'{vis_out_dir}/demo.mp4', embed=True)


In [ ]:
#@title now let's see how this works on a few different videos and discuss the results
videos_path = "./content/videos"
folder_dest = #path to your folder with videos on google drive
os.makedirs(videos_path, exist_ok=True)

!gdown --folder folder_dest --output {videos_path}

# load your data into the videos path

for video_file in os.listdir(videos_path):
    video_path = os.path.join(videos_path, video_file)

    !python ./demo/inferencer_demo.py {video_path} \
                --pose2d vitpose-h --pred-out-dir {pred_out_dir} \
                --vis-out-dir {vis_out_dir} --skeleton-style openpose --radius 5 --thickness 3 #--black-background